# Working with linear networks

Mink's *layers* work exactly as sklearn *transformers*, and linear networks work analogously to sklearn Pipelines.

To get a pipeline in mink, instead of using sklearn's `make_pipeline` with estimators and transformers, use mink's `make_network` with mink layers. Alternatively, define a network by hand.

## imports

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelBinarizer

In [2]:
from mink import layers
from mink import nonlinearities
from mink import objectives
from mink import updates
from mink import make_network

## load data

Use sklearn toy classification data.

In [3]:
X, y = make_classification(
    n_samples=2000,
    n_classes=5,
    n_informative=10,
    random_state=0,
)
y = LabelBinarizer().fit_transform(y)

## prepare network with `make_network`

We define a simple linear network with an input layer, a hidden layer, and an output layer, by passing a list of these layers to `make_network`.

In [4]:
layer_lst = [
    layers.InputLayer(),
    layers.DenseLayer(),
    layers.DenseLayer(
        num_units=5,
        nonlinearity=nonlinearities.Softmax(),
    ),
]

In [5]:
network = make_network(layer_lst)

## get symbolic output

In [6]:
Xs = tf.placeholder(dtype='float32', shape=(None, 20))

In [7]:
ys = tf.placeholder(dtype='float32', shape=(None, 5))

Like in sklearn, we can call `fit_transform` and the pipeline to get the output.

In [8]:
ys_out = network.fit_transform(Xs, ys)

## fit neural network

In [9]:
batch_size = 64

In [10]:
loss = objectives.CrossEntropy()(ys, ys_out)
train_step = updates.Momentum()(loss)
inputs = [train_step, loss]

In [11]:
init = tf.initialize_all_variables()
losses = []
with tf.Session() as session:
    session.run(init)
    for epoch in range(30):
        losses_epoch = []
        for i in range((X.shape[0] + batch_size - 1) // batch_size):
            Xb = X[i * batch_size:(i + 1) * batch_size]
            yb = y[i * batch_size:(i + 1) * batch_size]

            feed_dict = {Xs: Xb, ys: yb}
            _, loss = session.run(inputs, feed_dict=feed_dict)
            losses_epoch.append(loss)
        losses_mean = np.mean(losses_epoch)
        losses.append(losses_mean)
        print("Loss: {:.4f}".format(losses_mean))

Loss: 1.3625
Loss: 0.9353
Loss: 0.8366
Loss: 0.7714
Loss: 0.7194
Loss: 0.6756
Loss: 0.6371
Loss: 0.6030
Loss: 0.5730
Loss: 0.5455
Loss: 0.5204
Loss: 0.4980
Loss: 0.4777
Loss: 0.4597
Loss: 0.4428
Loss: 0.4278
Loss: 0.4135
Loss: 0.4010
Loss: 0.3889
Loss: 0.3780
Loss: 0.3674
Loss: 0.3572
Loss: 0.3481
Loss: 0.3389
Loss: 0.3307
Loss: 0.3232
Loss: 0.3155
Loss: 0.3081
Loss: 0.3016
Loss: 0.2951


## prepare network by hand

Alternatively, we can feedforward the output of each layer and pass the output to the next layer by hand. This will result in exactly the same network as defined above.

In [12]:
out = layers.InputLayer(Xs, ys).fit_transform(Xs)
out = layers.DenseLayer().fit_transform(out)
out = layers.DenseLayer(num_units=5, nonlinearity=nonlinearities.Softmax()).fit_transform(out)

In [13]:
loss = objectives.CrossEntropy()(ys, out)
train_step = updates.Momentum()(loss)
inputs = [train_step, loss]

In [14]:
init = tf.initialize_all_variables()
losses = []
with tf.Session() as session:
    session.run(init)
    for epoch in range(30):
        losses_epoch = []
        for i in range((X.shape[0] + batch_size - 1) // batch_size):
            Xb = X[i * batch_size:(i + 1) * batch_size]
            yb = y[i * batch_size:(i + 1) * batch_size]

            feed_dict = {Xs: Xb, ys: yb}
            _, loss = session.run(inputs, feed_dict=feed_dict)
            losses_epoch.append(loss)
        losses_mean = np.mean(losses_epoch)
        losses.append(losses_mean)
        print("Loss: {:.4f}".format(losses_mean))

Loss: 1.4079
Loss: 0.9245
Loss: 0.8336
Loss: 0.7704
Loss: 0.7192
Loss: 0.6755
Loss: 0.6367
Loss: 0.6030
Loss: 0.5731
Loss: 0.5461
Loss: 0.5218
Loss: 0.4997
Loss: 0.4801
Loss: 0.4622
Loss: 0.4456
Loss: 0.4304
Loss: 0.4168
Loss: 0.4040
Loss: 0.3920
Loss: 0.3812
Loss: 0.3702
Loss: 0.3608
Loss: 0.3514
Loss: 0.3424
Loss: 0.3342
Loss: 0.3262
Loss: 0.3185
Loss: 0.3115
Loss: 0.3049
Loss: 0.2982
